In [1]:
import gensim
# Download pretrained weights from https://code.google.com/archive/p/word2vec/
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)


In [8]:
import pandas as pd
data = pd.read_csv("aggregated_clean.csv")

len(data.index)

5865

In [9]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
similarities = []
for index, row in data.iterrows():
    target_pos = row["t_POS"].strip().split("|")
    prime_pos = row["p_POS"].strip().split("|")
    target = row["target"]
    prime = row["prime"]
    match = ""
    matched = False
    for t in target_pos:
        for p in prime_pos:
            if t == p:
                match = t
                matched = True
                break
        if matched:
            break
    s1 = None
    s2 = None
    if match == "NN":
        s1 = wn.synsets(prime, pos='n')
        s2 = wn.synsets(target, pos='n')
    elif match == "VB":
        s1 = wn.synsets(prime, pos='v')
        s2 = wn.synsets(target, pos='v')
    elif match == "RB":
        s1 = wn.synsets(prime, pos='r')
        s2 = wn.synsets(target, pos='r')
    elif match == "JJ":
        s1 = wn.synsets(row["prime"], pos='s')
        s2 = wn.synsets(row["target"], pos='s')
    elif match == "minor":
        #print("minor")
        #print(target)
        #print(prime)
        s1 = wn.synsets(row["prime"])
        s2 = wn.synsets(row["target"])
    else:
        similarities.append("NA")
        continue
    try:
        s = s1[0].jcn_similarity(s2[0], brown_ic)
        similarities.append(s)
    except:
        #print(target)
        #print(prime)
        similarities.append("NA")
data["jc_similarity"] = similarities
data

,Unnamed: 0,prime,target,RT.200SOA,RT.1200SOA,obs.200SOA,obs.1200SOA,primecond,rel,t_length,...,BEAGLE_RPs,BEAGLE_PMI,p_freq,p_logfreq,p_length,p_orthoN,p_bgsum,p_bgmean,p_POS,jc_similarity
0,4,ABILITY,summary,703.096774,785.545455,31,33,3,0,7,...,0.107813,0.073091,19.22,2.99,7.00,1.0,8681.0,1446.83,NN,0.0687943
1,5,ABILITY,capability,774.250000,908.838710,28,31,1,1,10,...,0.233508,0.488945,19.22,2.99,7.00,1.0,8681.0,1446.83,NN,0.701688
2,6,ABLE,beam,679.400000,787.571429,25,28,3,0,4,...,0.070516,0.039354,159.90,3.91,4.00,2.0,5602.0,1867.33,JJ,NA
3,7,ABLE,willing,833.821429,756.156250,28,32,1,1,7,...,0.355336,0.243430,159.90,3.91,4.00,2.0,5602.0,1867.33,JJ,NA
4,10,ABOVE,below,665.593750,592.718750,32,32,1,1,5,...,0.564871,0.779993,48.88,3.40,5.00,1.0,4212.0,1053.00,minor|JJ|NN,NA
5,11,ABOVE,property,662.580645,700.074074,31,27,3,0,8,...,0.021798,0.045033,48.88,3.40,5.00,1.0,4212.0,1053.00,minor|JJ|NN,1e-300
6,12,ABSTRACT,unknown,669.709677,672.093750,31,32,4,0,7,...,0.034185,0.078829,8.00,2.18,2.05,0.0,12196.0,1742.29,JJ|NN|VB,NA
7,13,ABSTRACT,thought,594.939394,716.645161,33,31,2,1,7,...,0.121965,0.328407,8.00,2.18,2.05,0.0,12196.0,1742.29,JJ|NN|VB,0.166715
8,14,ABSURD,ridiculous,783.266667,679.774193,30,31,1,1,10,...,0.159454,6.306579,9.71,2.70,6.00,0.0,4072.0,814.40,JJ|NN,NA
9,15,ABSURD,satin,947.464286,936.291667,28,24,3,0,5,...,0.099744,0.112832,9.71,2.70,6.00,0.0,4072.0,814.40,JJ|NN,0.0411417


In [10]:
# try path similarities
second_similarities = []
for index, row in data.iterrows():
    target = row["target"]
    prime = row["prime"]
    s1 = wn.synsets(target)[0]
    s2 = wn.synsets(prime)[0]
    try:
        ws = s1.path_similarity(s2)
        second_similarities.append(ws)
    except:
        print("yikes")
        second_similarities.append("NA")
data["path_similarity"] = second_similarities
    

In [11]:
from scipy import spatial
# get the cosine distances for embeddings
# model[word] to get the embedding
cosine_distances = []
for index, row in data.iterrows():
    target = row["target"]
    prime = row["prime"]
    tm = model[target]
    pm = model[prime]
    cosine_dist = spatial.distance.cosine(tm, pm)
    cosine_distances.append(cosine_dist)
data["cosine_distance"] = cosine_distances
data

,Unnamed: 0,prime,target,RT.200SOA,RT.1200SOA,obs.200SOA,obs.1200SOA,primecond,rel,t_length,...,p_freq,p_logfreq,p_length,p_orthoN,p_bgsum,p_bgmean,p_POS,jc_similarity,path_similarity,cosine_distance
0,4,ABILITY,summary,703.096774,785.545455,31,33,3,0,7,...,19.22,2.99,7.00,1.0,8681.0,1446.83,NN,0.0687943,0.125000,1.003616
1,5,ABILITY,capability,774.250000,908.838710,28,31,1,1,10,...,19.22,2.99,7.00,1.0,8681.0,1446.83,NN,0.701688,0.500000,0.832655
2,6,ABLE,beam,679.400000,787.571429,25,28,3,0,4,...,159.90,3.91,4.00,2.0,5602.0,1867.33,JJ,NA,NaN,0.963301
3,7,ABLE,willing,833.821429,756.156250,28,32,1,1,7,...,159.90,3.91,4.00,2.0,5602.0,1867.33,JJ,NA,NaN,0.995506
4,10,ABOVE,below,665.593750,592.718750,32,32,1,1,5,...,48.88,3.40,5.00,1.0,4212.0,1053.00,minor|JJ|NN,NA,NaN,0.761463
5,11,ABOVE,property,662.580645,700.074074,31,27,3,0,8,...,48.88,3.40,5.00,1.0,4212.0,1053.00,minor|JJ|NN,1e-300,0.111111,0.928554
6,12,ABSTRACT,unknown,669.709677,672.093750,31,32,4,0,7,...,8.00,2.18,2.05,0.0,12196.0,1742.29,JJ|NN|VB,NA,0.076923,0.970089
7,13,ABSTRACT,thought,594.939394,716.645161,33,31,2,1,7,...,8.00,2.18,2.05,0.0,12196.0,1742.29,JJ|NN|VB,0.166715,0.333333,1.081407
8,14,ABSURD,ridiculous,783.266667,679.774193,30,31,1,1,10,...,9.71,2.70,6.00,0.0,4072.0,814.40,JJ|NN,NA,NaN,0.618680
9,15,ABSURD,satin,947.464286,936.291667,28,24,3,0,5,...,9.71,2.70,6.00,0.0,4072.0,814.40,JJ|NN,0.0411417,0.083333,0.804199


In [12]:
data.to_csv("distances_data.csv")

5865